In [1]:
# ps51
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9bfeda7aa499f77202895dd6d97bd743272f31d19c78181af5fd018f98b3fb51
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


🌟 𝐏𝐘𝐒𝐏𝐀𝐑𝐊 𝐏𝐫𝐚𝐜𝐭𝐢𝐜𝐞/𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 📊
========================================
𝐈𝐦𝐦𝐞𝐝𝐢𝐚𝐭𝐞 𝐅𝐨𝐨𝐝 𝐃𝐞𝐥𝐢𝐯𝐞𝐫𝐲 𝐈 𝐏𝐫𝐨𝐛𝐥𝐞𝐦

Write an Pyspark code to find the percentage of immediate orders in the table, rounded to 2 decimal places.
If the preferred delivery date of the customer is the same as the order date then the order is called immediate otherwise it’s called scheduled.

Schema And Data:
================
 # Define the schema for the DataFrame
schema = StructType([
 StructField("delivery_id", IntegerType(), True),
 StructField("customer_id", IntegerType(), True),
 StructField("order_date", StringType(), True),
 StructField("customer_pref_delivery_date", StringType(), True),
])
# Create the data
data = [
 (1, 1, "2019-08-01", "2019-08-02"),
 (2, 5, "2019-08-02", "2019-08-02"),
 (3, 1, "2019-08-11", "2019-08-11"),
 (4, 3, "2019-08-24", "2019-08-26"),
 (5, 4, "2019-08-21", "2019-08-22"),
 (6, 2, "2019-08-11", "2019-08-13"),
]

In [2]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# make spark session
spark = SparkSession.builder.master('local').appName('pyspark-day-6').getOrCreate()

In [4]:
spark

In [5]:
# define schema and data for df
schema = StructType([
 StructField("delivery_id", IntegerType(), True),
 StructField("customer_id", IntegerType(), True),
 StructField("order_date", StringType(), True),
 StructField("customer_pref_delivery_date", StringType(), True),
])
data = [
 (1, 1, "2019-08-01", "2019-08-02"),
 (2, 5, "2019-08-02", "2019-08-02"),
 (3, 1, "2019-08-11", "2019-08-11"),
 (4, 3, "2019-08-24", "2019-08-26"),
 (5, 4, "2019-08-21", "2019-08-22"),
 (6, 2, "2019-08-11", "2019-08-13"),
]

In [6]:
# create df
order_df = spark.createDataFrame(schema = schema, data = data)
order_df.show()

+-----------+-----------+----------+---------------------------+
|delivery_id|customer_id|order_date|customer_pref_delivery_date|
+-----------+-----------+----------+---------------------------+
|          1|          1|2019-08-01|                 2019-08-02|
|          2|          5|2019-08-02|                 2019-08-02|
|          3|          1|2019-08-11|                 2019-08-11|
|          4|          3|2019-08-24|                 2019-08-26|
|          5|          4|2019-08-21|                 2019-08-22|
|          6|          2|2019-08-11|                 2019-08-13|
+-----------+-----------+----------+---------------------------+



In [7]:
order_df.printSchema()

root
 |-- delivery_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_pref_delivery_date: string (nullable = true)



In [8]:
# add col as is_immediate
order_df = order_df.withColumn('is_immediate', when(col('order_date') == col('customer_pref_delivery_date'), 1).otherwise(0))
order_df.show()

+-----------+-----------+----------+---------------------------+------------+
|delivery_id|customer_id|order_date|customer_pref_delivery_date|is_immediate|
+-----------+-----------+----------+---------------------------+------------+
|          1|          1|2019-08-01|                 2019-08-02|           0|
|          2|          5|2019-08-02|                 2019-08-02|           1|
|          3|          1|2019-08-11|                 2019-08-11|           1|
|          4|          3|2019-08-24|                 2019-08-26|           0|
|          5|          4|2019-08-21|                 2019-08-22|           0|
|          6|          2|2019-08-11|                 2019-08-13|           0|
+-----------+-----------+----------+---------------------------+------------+



In [12]:
# apply agg
order_df.agg(round((sum('is_immediate')/ count("*")*100),2).alias('immediate_%')).show()

+-----------+
|immediate_%|
+-----------+
|      33.33|
+-----------+

